# Semantic Search with Metadata Filtering using FAISS and Amazon Bedrock
> *If you see errors, you may need to be allow-listed for the Bedrock models used by this notebook*

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*


## Install dependencies

This dependencies script in the root of this repo provides a set of boto3 and cli resources to allow for Bedrock use while in preview.

This notebook demonstrates invoking Bedrock models directly using the AWS SDK, but for later notebooks in the workshop you'll also need to install [LangChain](https://github.com/hwchase17/langchain):

In [2]:
%pip install langchain==0.0.309 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
botocore 1.32.6 requires urllib3<2.1,>=1.25.4; python_version >= "3.10", but you have urllib3 2.1.0 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
hdijupyterutils 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.2 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.2 which is incompatible.


In [3]:
%pip install pydantic==1.10.13 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.0 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.16.1 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.1 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install sqlalchemy==2.0.21 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.0 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.16.1 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.1 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In this example, you will use [Facebook AI Similarity Search (Faiss)](https://faiss.ai/) as the vector database to store your embeddings. There are CPU or GPU options available, depending on your platform.

In [5]:
%pip install faiss-cpu==1.7.4 # For CPU Installation
#%pip install faiss-gpu # For CUDA 7.5+ Supported GPU's.

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In this example, you will use several years of Amazon's Letter to Shareholders as a text corpus to perform Q&A on.

First you will download these files from the internet.

In [6]:
%pip install pypdf==3.14.0

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
!mkdir -p ./data

from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2020-Shareholder-Letter.pdf',
    'AMZN-2019-Shareholder-Letter.pdf'
]

metadata = [
    dict(year=2022, source=filenames[0]),
    dict(year=2021, source=filenames[1]),
    dict(year=2020, source=filenames[2]),
    dict(year=2019, source=filenames[3])]

data_root = "./data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

As part of Amazon's culture, the CEO always includes a copy of the 1997 Letter to Shareholders with every new release. This will cause repetition, take longer to generate embeddings, and may skew your results. In the next section you will take the downloaded data, trim the 1997 letter (last 3 pages) and overwrite them as processed files.

In [10]:
import glob
from pypdf import PdfReader, PdfWriter

local_pdfs = glob.glob(data_root + '*.pdf')

for local_pdf in local_pdfs:
    pdf_reader = PdfReader(local_pdf)
    pdf_writer = PdfWriter()
    for pagenum in range(len(pdf_reader.pages)-3):
        page = pdf_reader.pages[pagenum]
        pdf_writer.add_page(page)

    with open(local_pdf, 'wb') as new_file:
        new_file.seek(0)
        pdf_writer.write(new_file)
        new_file.truncate()


Now that you have clean PDFs to work with, you will enrich your documents with metadata, then use a process called "chunking" to break up a larger document into small pieces. These small pieces will allow you to generate embeddings without surpassing the input limit of the embedding model.

In this example you will break the document into 1000 character chunks, with a 100 character overlap. This will allow your embeddings to maintain some of its context.

In [11]:
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

documents = []

for idx, file in enumerate(filenames):
    loader = PyPDFLoader(data_root + file)
    document = loader.load()
    for document_fragment in document:
        document_fragment.metadata = metadata[idx]
        
    print(f'{len(document)} {document}\n')
    documents += document

# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
)

docs = text_splitter.split_documents(documents)

4 [Document(page_content='Dear shareholders:\nAs I sit down to write my second annual shareholder letter as CEO, I find myself optimistic and energized\nby what lies ahead for Amazon. Despite 2022 being one of the harder macroeconomic years in recent memory,and with some of our own operating challenges to boot, we still found a way to grow demand (on top ofthe unprecedented growth we experienced in the first half of the pandemic). We innovated in our largestbusinesses to meaningfully improve customer experience short and long term. And, we made importantadjustments in our investment decisions and the way in which we’ll invent moving forward, while stillpreserving the long-term investments that we believe can change the future of Amazon for customers,\nshareholders, and employees.\nWhile there were an unusual number of simultaneous challenges this past year, the reality is that if you\noperate in large, dynamic, global market segments with many capable and well-funded competitors (theco

## Create the boto3 client

Interaction with the Bedrock API is done via boto3 SDK. To create a the Bedrock client, we are providing an utility method that supports different options for passing credentials to boto3. 
If you are running these notebooks from your own computer, make sure you have [installed the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html) before proceeding.

#### Use default credential chain

If you are running this notebook from a Sagemaker Studio notebook and your Sagemaker Studio role has permissions to access Bedrock you can just run the cells below as-is. This is also the case if you are running these notebooks from a computer whose default credentials have access to Bedrock

#### Use a different role

In case you or your company has setup a specific role to access Bedrock, you can specify such role by uncommenting the line `#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'` in the cell below before executing it. Ensure that your current user or role have permissions to assume such role.

#### Use a specific profile

In case you are running this notebooks from your own computer and you have setup the AWS CLI with multiple profiles and the profile which has access to Bedrock is not the default one, you can uncomment the line `#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'` and specify the profile to use.

#### Note about `langchain`

The Bedrock classes provided by `langchain` create a default Bedrock boto3 client. We recommend to explicitly create the Bedrock client using the instructions below, and pass it to the class instantiation methods using `client=bedrock_runtime`

In [12]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

In [13]:
import boto3
import json 

bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

## Building a FAISS vector database

In this example, you will be using the Amazon Titan Embeddings Model from Amazon Bedrock to generate the embeddings for our FAISS vector database.

The `TokenCounterHandler` callback function is a function you can utilize in your LLM objects and chains to generate reports on token count. It is supplied here as a utility class that will output the token counts at the end of your result chain, or can be attached to a LLM object and invoked manually.

In [14]:
%pip install tiktoken==0.4.0 --force-reinstall

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.3 MB/s eta 0:00:00
  Obtaining dependency information for regex>=2022.1.18 from https://files.pythonhosted.org/packages/8f/3e/4b8b40eb3c80aeaf360f0361d956d129bb3d23b2a3ecbe3a04a8f3bdd6d3/regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 210.4 MB/s eta 0:00:00
  Obtaining dependency information for requests>=2.26.0 from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/da/f1/3702ba2a7470666a62fd81c58a4c40be00670e5006a67f4d626e57f013ae/charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information f

In [15]:
from utils.TokenCounterHandler import TokenCounterHandler

token_counter = TokenCounterHandler()

In [16]:
from langchain.embeddings import BedrockEmbeddings

embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",
                               client=bedrock_runtime)

Next you will import the Document and FAISS modules from Langchain. Using these modules will allow you to quickly generate embeddings through Amazon Bedrock and store them locally in your FAISS vector store.

In [17]:
from langchain.schema import Document
from langchain.vectorstores import FAISS

In this step you will process documents and prepare them to be converted to vectors for the vector store.

Here you will use the from_documents function in the Langchain FAISS provider to build a vector database from your document embeddings.

In [18]:
db = FAISS.from_documents(docs, embeddings)

To avoid having to completely regenerate your embeddings all the time, you can save and load the vector store from the local filesystem. In the next section you will save the vector store locally, and reload it.

In [19]:
db.save_local("faiss_titan_index")

In [20]:
new_db = FAISS.load_local("faiss_titan_index", embeddings)

In [21]:
db = new_db

## Similarity Searching

Here you will set your search query, and look for documents that match.

In [22]:
query = "How has AWS evolved?"

### Basic Similarity Search

The results that come back from the `similarity_search_with_score` API are sorted by score from lowest to highest. The score value is represented by the [L-squared (or L2)](https://en.wikipedia.org/wiki/Lp_space) distance of each result. Lower scores are better, repesenting a shorter distance between vectors.

In [23]:
results_with_scores = db.similarity_search_with_score(query)
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}\nScore: {score}\n\n")

Content: AWS : As we were defining AWS and working backwards on the services we thought customers wanted, we
kept triggering one of the biggest tensions in product development—where to draw the line on functionality inV1. One early meeting in particular—for our core compute service called Elastic Compute Cloud (“EC2”)—was scheduled for an hour, and took three, as we animatedly debated whether we could launch a computeservice without an accompanying persistent block storage companion (a form of network attached storage).
Metadata: {'year': 2021, 'source': 'AMZN-2021-Shareholder-Letter.pdf'}
Score: 160.02227783203125


Content: still required substantial capital investment. There were voicesinside and outside of the company questioning why Amazon (known mostly as an online retailer then) wouldbe investing so much in cloud computing. But, we knew we were inventing something special that couldcreate a lot of value for customers and Amazon in the future. We had a head start on potential com

### Similarity Search with Metadata Filtering
Additionally, you can provide metadata to your query to filter the scope of your results. The `filter` parameter for search queries is a dictionary of metadata key/value pairs that will be matched to results to include/exclude them from your query.

In [24]:
filter = dict(year=2022)

In the next section, you will notice that your query has returned less results than the basic search, because of your filter criteria on the resultset.

In [25]:
results_with_scores = db.similarity_search_with_score(query, filter=filter)
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}, Score: {score}\n\n")

Content: still required substantial capital investment. There were voicesinside and outside of the company questioning why Amazon (known mostly as an online retailer then) wouldbe investing so much in cloud computing. But, we knew we were inventing something special that couldcreate a lot of value for customers and Amazon in the future. We had a head start on potential competitors;and if anything, we wanted to accelerate our pace of innovation. We made the long-term decision tocontinue investing in AWS. Fifteen years later, AWS is now an $85B annual revenue run rate business, withstrong profitability, that has transformed how customers from start-ups to multinational companies to publicsector organizations manage their technology infrastructure. Amazon would be a different company ifwe’d slowed investment in AWS during that 2008-2009 period.
Metadata: {'year': 2022, 'source': 'AMZN-2022-Shareholder-Letter.pdf'}, Score: 166.88641357421875


Content: When I joined Amazon in 1997, we had 

### Top-K Matching

Top-K Matching is a filtering technique that involves a 2 stage approach.

1. Perform a similarity search, returning the top K matches.
2. Apply your metadata filter on the smaller resultset.

Note: A caveat for Top-K matching is that if the value for K is too small, there is a chance that after filtering there will be no results to return.

Using Top-K matching requires 2 values:
- `k`, the max number of results to return at the end of our query
- `fetch_k`, the max number of results to return from the similarity search before applying filters


In [26]:
results = db.similarity_search(query, filter=filter, k=2, fetch_k=4)
for doc in results:
    print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}\n\n")

Content: still required substantial capital investment. There were voicesinside and outside of the company questioning why Amazon (known mostly as an online retailer then) wouldbe investing so much in cloud computing. But, we knew we were inventing something special that couldcreate a lot of value for customers and Amazon in the future. We had a head start on potential competitors;and if anything, we wanted to accelerate our pace of innovation. We made the long-term decision tocontinue investing in AWS. Fifteen years later, AWS is now an $85B annual revenue run rate business, withstrong profitability, that has transformed how customers from start-ups to multinational companies to publicsector organizations manage their technology infrastructure. Amazon would be a different company ifwe’d slowed investment in AWS during that 2008-2009 period.
Metadata: {'year': 2022, 'source': 'AMZN-2022-Shareholder-Letter.pdf'}


Content: When I joined Amazon in 1997, we had booked $15M in revenue in 1

### Maximal Marginal Relevance

Another measurement of results is Maximal Marginal Relevance (MMR). The focus of MMR is to minimize the redundancy of your search results while still maintaining relevance by re-ranking the results to provide both similarity and diversity.

In the next section you will use the `max_marginal_relevance_search` API to run the same query as in the Metadata Filtering section, but with reranked results.

In [27]:
results = db.max_marginal_relevance_search(query, filter=filter)
for doc in results:
    print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}\n\n")

Content: still required substantial capital investment. There were voicesinside and outside of the company questioning why Amazon (known mostly as an online retailer then) wouldbe investing so much in cloud computing. But, we knew we were inventing something special that couldcreate a lot of value for customers and Amazon in the future. We had a head start on potential competitors;and if anything, we wanted to accelerate our pace of innovation. We made the long-term decision tocontinue investing in AWS. Fifteen years later, AWS is now an $85B annual revenue run rate business, withstrong profitability, that has transformed how customers from start-ups to multinational companies to publicsector organizations manage their technology infrastructure. Amazon would be a different company ifwe’d slowed investment in AWS during that 2008-2009 period.
Metadata: {'year': 2022, 'source': 'AMZN-2022-Shareholder-Letter.pdf'}


Content: general-purpose CPU processors named Graviton. Graviton2-based c

## Q&A with Anthropic Claude and Retrieved Vectors

Now that you are able to query from the vector store, you're ready to feed context into your LLM.

Using the LangChain wrapper for Bedrock, creating an object for the LLM can be done in a single line of code where you specify the model_id of the desired LLM (Claude V2 in this case), and any model level arguments.

In [28]:
from langchain.llms.bedrock import Bedrock

model_kwargs_titan = { 
    "maxTokenCount": 512,
    "stopSequences": [],
    "temperature":0.0,  
    "topP":0.5
}

# Amazon Titan Model
llm = Bedrock(
    model_id="amazon.titan-text-express-v1", 
    client=bedrock_runtime, 
    model_kwargs=model_kwargs_titan,
    callbacks=[token_counter]
)

Since you have a model object set up, you can use it to get a baseline of what the LLM will produce without any provided context.

Something you will notice is with the prompt "How has AWS evolved?", the answer isn't bad, but its not exactly what you'd look for from the lens of an executive. You'd want to hear about how they approached things that led to evolution, whereas the baseline results are just facts that indicate change. Later in the notebook, you will provide context to get a more tailored answer.

In [29]:
print(llm.predict("How has AWS evolved?"))

token_counter.report()


Here are some ways in which AWS has evolved:
1. AWS has expanded its geographical presence by establishing data centers and regions worldwide. This allows customers to deploy their applications and services closer to their users, reducing latency and improving performance.
2. AWS has continuously added new services and features to its portfolio. This includes machine learning, artificial intelligence, blockchain, serverless computing, and more. By offering a wide range of services, AWS enables customers to build and deploy complex applications and solutions.
3. AWS has invested heavily in security and compliance. This includes implementing robust security measures, such as encryption, identity and access management, and compliance with industry standards and regulations. AWS has also earned numerous certifications, such as ISO 27001, PCI DSS, and HIPAA, to demonstrate its commitment to security.
4. AWS has a strong focus on customer satisfaction and innovation. This includes providing

With your LLM ready to go, you'll create a prompt template to utilize context to answer a given question. Prompt formats will be different by model, so if you change your model you will also likely need to adjust your prompt.

In [30]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """

Human: Here is a set of context, contained in <context> tags:

<context>
{context}
</context>

Use the context to provide an answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{question}

Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

With the LLM endpoint object created, you are ready to create your first chain!

This chain is a simple example using LangChain's RetrievalQA chain, which will:
- take a query as input
- generate query embeddings
- query the vector database for relevant document chunks based on the query embedding
- inject the context and original query into the prompt template
- invoke the LLM with the completed prompt
- return the LLM result

The [`stuff` chain type](https://python.langchain.com/docs/modules/chains/document/stuff) simply takes the context documents and inserts them into the prompt.

By setting `return_source_documents` to `True`, the LLM responses will also contain the document chunks from the vector database, to illustrate where the context came from.

In [31]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5, "filter": filter},
        callbacks=[token_counter]
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},
    callbacks=[token_counter]
)

Now that your chain is set up, you can supply queries to it and generate responses based on your source documents.

You'll note that the LLM response references the context documents provided, using them to formulate a response calling out things that were mentioned specifically by Amazon's CEO.

In [32]:
query = "How has AWS evolved?"
result = qa({"query": query})

print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')


Token Counts:
Total: 1399
Embedding: N/A
Prompt: 1006
Generation:393

Query: How has AWS evolved?

Result:  AWS has evolved from a books-only retailer in 1996 to a global online retailer and cloud computing leader, with a diverse portfolio of services and a strong focus on innovation and customer satisfaction.

Context Documents: 
page_content='still required substantial capital investment. There were voicesinside and outside of the company questioning why Amazon (known mostly as an online retailer then) wouldbe investing so much in cloud computing. But, we knew we were inventing something special that couldcreate a lot of value for customers and Amazon in the future. We had a head start on potential competitors;and if anything, we wanted to accelerate our pace of innovation. We made the long-term decision tocontinue investing in AWS. Fifteen years later, AWS is now an $85B annual revenue run rate business, withstrong profitability, that has transformed how customers from start-ups to

In [33]:
query = "Why is Amazon successful?"
result = qa({"query": query})

print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')


Token Counts:
Total: 2449
Embedding: N/A
Prompt: 1703
Generation:746

Query: Why is Amazon successful?

Result:  Amazon's Advertising business is uniquely effective for brands, which is part of why it
continues to grow at a brisk clip. Akin to physical retailers’ advertising businesses selling shelf space, end-
caps, and placement in their circulars, our sponsored products and brands offerings have been an integral part

We also took this occasion to make larger structural changes that set us up better to deliver lower costs and
faster speed for many years to come. A good example was reevaluating how our US fulfillment network wasorganized. Until recently, Amazon operated one national US fulfillment network that distributed inventoryfrom fulfillment centers spread across the entire country. If a local fulfillment center didn’t have the product acustomer ordered, we’d end up shipping it from other parts of the country, costing us more and increasingdelivery times. This challenge became

In [34]:
query = "What business challenges has Amazon experienced?"
result = qa({"query": query})

print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')


Token Counts:
Total: 3189
Embedding: N/A
Prompt: 2414
Generation:775

Query: What business challenges has Amazon experienced?

Result:  rising cost to serve in our Stores fulfillment network, and making several changes that we believe will meaningfully improve our fulfillment costs and speed of delivery.


Context Documents: 
page_content='operate in large, dynamic, global market segments with many capable and well-funded competitors (theconditions in which Amazon operates all of its businesses), conditions rarely stay stagnant for long.\nIn the 25 years I’ve been at Amazon, there has been constant change, much of which we’ve initiated ourselves.' metadata={'year': 2022, 'source': 'AMZN-2022-Shareholder-Letter.pdf'}

page_content='We also took this occasion to make larger structural changes that set us up better to deliver lower costs and\nfaster speed for many years to come. A good example was reevaluating how our US fulfillment network wasorganized. Until recently, Amazon operated o

In [35]:
query = "How was Amazon impacted by COVID-19?"
result = qa({"query": query})

print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')


Token Counts:
Total: 4116
Embedding: N/A
Prompt: 3153
Generation:963

Query: How was Amazon impacted by COVID-19?

Result:  Amazon's consumer business grew during the early part of the pandemic, as many physical stores shut down. The company also took this opportunity to make larger structural changes that set it up better to deliver lower costs and faster speed for many years to come. A good example was reevaluating how Amazon's US fulfillment network was organized. Until recently, Amazon operated one national US fulfillment network that distributed inventory from fulfillment centers spread across the entire country. If a local fulfillment center didn't have the product a customer ordered, we'd end up shipping it from other parts of the country, costing us more and increasing delivery times. This challenge became more pronounced as Amazon's fulfillment network expanded to hundreds of additional nodes over the last few years, distributing inventory across more locations and increasing